In [1]:
import pandas as pd
import tensorflow as tf 
import tensorflow_io as tfio
pd.options.mode.chained_assignment = None

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
check_df = pd.read_csv('/kaggle/input/new-sound-dataset-2024/df_sound_super_ext1.csv')

In [4]:
df = pd.read_csv('/kaggle/input/new-sound-dataset-2024/df_sound_super_ext1.csv')
df.head(5)

,label,MFCCs_0,MFCCs_1,MFCCs_2,MFCCs_3,MFCCs_4,MFCCs_5,MFCCs_6,MFCCs_7,MFCCs_8,...,spectral_contrast_std_3,spectral_contrast_std_4,spectral_contrast_std_5,spectral_contrast_std_6,tonnetz_std_0,tonnetz_std_1,tonnetz_std_2,tonnetz_std_3,tonnetz_std_4,tonnetz_std_5
0,Truck idling Full,-144.266110,147.17151,-49.638912,15.333573,-8.378953,25.929523,-9.700624,12.629977,-24.272520,...,1.907221,1.448494,1.114312,6.783267,0.037209,0.025670,0.027315,0.044734,0.012590,0.014134
1,Truck idling Full,-155.357360,144.67230,-46.065895,18.556850,11.784542,24.411234,-3.243358,20.571308,0.615909,...,1.986655,1.726565,1.379506,4.743701,0.034316,0.033457,0.030612,0.032710,0.018051,0.014341
2,Truck idling Full,-135.089160,120.50888,-21.998446,20.289173,-16.049006,5.310522,-1.421212,6.153967,-8.638702,...,2.033297,1.786614,1.418294,5.573070,0.030168,0.024719,0.032026,0.027985,0.012248,0.014188
3,Truck idling Full,-145.972380,145.63855,-56.518494,14.895403,-5.488753,26.655867,-5.340638,15.659369,-7.542261,...,1.861915,1.680943,1.353896,6.479958,0.029149,0.026572,0.025946,0.029970,0.013266,0.013706
4,Truck idling Full,-115.146996,158.91006,-58.141830,9.406412,-24.740862,30.780817,-12.989754,14.106170,-11.900005,...,2.389258,1.740846,1.157450,7.152780,0.050403,0.065989,0.081200,0.058970,0.032351,0.031634


In [5]:
df['true_number'] = df.index
last_label = 'kekw'
k = 1
for i in range(df.shape[0]):
    if df['label'].loc[i] != last_label:
        k = 1
    df['true_number'].loc[i] = k
    last_label = df['label'].loc[i]
    k += 1

In [6]:
df.label = df.label.apply(lambda x: 'Tram' if x in ['Tram Acceleration Full','Tram Full', 'Tram Ring Full', 'Tram Braking Full'] else x)

In [7]:
df['true_label'] = check_df['label']

In [8]:
def convert_dataset(df1):
    df2 = df1.copy()
    labels = list(zip(list(df2['label'].unique()), range(17)))
    my_dict = {sublist[0]: sublist[1] for sublist in labels}
    df2['label'] = df2.label.apply(lambda x: my_dict[x])
    columns_to_convert = [
    'MFCCs_0', 'MFCCs_1', 'MFCCs_2', 'MFCCs_3', 'MFCCs_4', 'MFCCs_5', 'MFCCs_6', 'MFCCs_7', 'MFCCs_8', 'MFCCs_9',
    'MFCCs_10', 'MFCCs_11', 'MFCCs_12', 'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18', 'MFCCs_19',
    'MFCCs_20', 'MFCCs_21', 'MFCCs_22', 'MFCCs_23', 'MFCCs_24', 'MFCCs_25', 'MFCCs_26', 'MFCCs_27', 'MFCCs_28', 'MFCCs_29',
    'MFCCs_30', 'MFCCs_31', 'MFCCs_32', 'MFCCs_33', 'MFCCs_34', 'MFCCs_35', 'MFCCs_36', 'MFCCs_37', 'MFCCs_38', 'MFCCs_39',
    'Chroma_0', 'Chroma_1', 'Chroma_2', 'Chroma_3', 'Chroma_4', 'Chroma_5', 'Chroma_6', 'Chroma_7', 'Chroma_8', 'Chroma_9',
    'Chroma_10', 'Chroma_11', 'Mel_0', 'Mel_1', 'Mel_2', 'Mel_3', 'Mel_4', 'Mel_5', 'Mel_6', 'Mel_7', 'Mel_8', 'Mel_9',
    'Mel_10', 'Mel_11', 'Mel_12', 'Mel_13', 'Mel_14', 'Mel_15', 'Mel_16', 'Mel_17', 'Mel_18', 'Mel_19', 'Mel_20', 'Mel_21',
    'Mel_22', 'Mel_23', 'Mel_24', 'Mel_25', 'Mel_26', 'Mel_27', 'Mel_28', 'Mel_29', 'Mel_30', 'Mel_31', 'Mel_32', 'Mel_33',
    'Mel_34', 'Mel_35', 'Mel_36', 'Mel_37', 'Mel_38', 'Mel_39', 'Mel_40', 'Mel_41', 'Mel_42', 'Mel_43', 'Mel_44', 'Mel_45',
    'Mel_46', 'Mel_47', 'Mel_48', 'Mel_49', 'Mel_50', 'Mel_51', 'Mel_52', 'Mel_53', 'Mel_54', 'Mel_55', 'Mel_56', 'Mel_57',
    'Mel_58', 'Mel_59', 'Mel_60', 'Mel_61', 'Mel_62', 'Mel_63', 'Mel_64', 'Mel_65', 'Mel_66', 'Mel_67', 'Mel_68', 'Mel_69',
    'Mel_70', 'Mel_71', 'Mel_72', 'Mel_73', 'Mel_74', 'Mel_75', 'Mel_76', 'Mel_77', 'Mel_78', 'Mel_79', 'Mel_80', 'Mel_81',
    'Mel_82', 'Mel_83', 'Mel_84', 'Mel_85', 'Mel_86', 'Mel_87', 'Mel_88', 'Mel_89', 'Mel_90', 'Mel_91', 'Mel_92', 'Mel_93',
    'Mel_94', 'Mel_95', 'Mel_96', 'Mel_97', 'Mel_98', 'Mel_99', 'Mel_100', 'Mel_101', 'Mel_102', 'Mel_103', 'Mel_104',
    'Mel_105', 'Mel_106', 'Mel_107', 'Mel_108', 'Mel_109', 'Mel_110', 'Mel_111', 'Mel_112', 'Mel_113', 'Mel_114', 'Mel_115',
    'Mel_116', 'Mel_117', 'Mel_118', 'Mel_119', 'Mel_120', 'Mel_121', 'Mel_122', 'Mel_123', 'Mel_124', 'Mel_125', 'Mel_126',
    'Mel_127', 'Contrast_0', 'Contrast_1', 'Contrast_2', 'Contrast_3', 'Contrast_4', 'Contrast_5', 'Contrast_6', 'Tonnetz_0',
    'Tonnetz_1', 'Tonnetz_2', 'Tonnetz_3', 'Tonnetz_4', 'Tonnetz_5'
    ]

    # Convert specified columns to float64
    df2[columns_to_convert] = df2[columns_to_convert].astype('float64')
    
    return df2, my_dict

In [9]:
df, names = convert_dataset(df)
df.groupby('label').count().T.head(1)

label,0,1,2,3,4,5,6,7,8,9,10,11,12,13
MFCCs_0,346,485,210,1231,240,492,204,208,471,296,201,304,486,200


In [10]:
names_rev = {j:i for i, j in names.items()}
names_rev

{0: 'Truck idling Full',
 1: 'Siren 4 Full',
 2: 'Car Idling Full',
 3: 'Tram',
 4: 'Truck Acceleration Full',
 5: 'Moto Acceleration Full',
 6: 'Car Horn Full',
 7: 'Truck Horn Full',
 8: 'Siren 5 Full',
 9: 'Truck Braking Full',
 10: 'Car acceleration Full',
 11: 'Moto Idling Full',
 12: 'Siren 1 Full',
 13: 'Car braking Full'}

In [11]:
df_to_train = df#.drop(imp[imp.Importance < 0].Feature, axis=1)
df.shape, df_to_train.shape

((5374, 389), (5374, 389))

In [12]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

df_to_train = df_to_train

target_column = 'label' 
X = df_to_train.drop(target_column, axis=1)
y = df_to_train[target_column]

# Трейн-тест сплит
X_train_, X_test_, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train_.drop(['true_label', 'true_number'], axis=1)
X_test = X_test_.drop(['true_label', 'true_number'], axis=1)
params = {'iterations': 671,
           'learning_rate': 0.09266820440668619,
           'depth': 6,
           'l2_leaf_reg': 0.015597347894183615,
           'border_count': 128,
           'bagging_temperature': 0.012884215330488278,
           'random_strength': 0.09895330797524414
         }
model = CatBoostClassifier(**params)
model.fit(X_train, y_train, verbose=False)


In [14]:
model.save_model('cat_model.cdm')